# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [15]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
import joblib

from azureml.widgets import RunDetails
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.pipeline.steps import AutoMLStep
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

This dataset contains California Housing Price data downloaded from https://raw.githubusercontent.com/ageron/handson-ml/master/ and was used as an example in Hands-On Machine Learning with Scikit-Learn & TensorFlow by Aurelien Geron. 

This dataset is an adapted version from the original data from the StatLib repository collected from the 1990 California census. 

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'California-housing-price-projection'
experiment=Experiment(ws, experiment_name)

quick-starts-ws-137405
aml-quickstarts-137405
southcentralus
48a74bb7-9950-4cc1-9caa-5d50f995cc55


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
#create a compute target
amlcompute_cluster_name = "compute-housing"

# Verify that cluster does not exist already, if not create a new one with amlcompute_cluster_name
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           vm_priority = 'lowpriority', 
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 0, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [7]:
#check if the dataset exists and register the dataset if not
import train
found = False
key = "California_housing_price-1"
description_text = "California housing price from 1990 census"

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]
    print("found dataset") 

if not found:       
        #If the dataset doesn't exist, then download data and register the Dataset in Workspace
#        DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
#        HOUSING_PATH = "datasets/housing"
#        HOUSING_URL = DOWNLOAD_ROOT + HOUSING_PATH + "/housing.tgz"
#        df = train.fetch_housing_data(HOUSING_URL, HOUSING_PATH)
    path = r"https://github.com/second-husky/Capstone-Azure-ML/raw/master/starter_file/housing.csv"
    datastore = Datastore(ws, name = None)
    df = pd.read_csv(path)
    print(df.head(5))
    dataset = TabularDatasetFactory.register_pandas_dataframe(df, datastore, 'housing_data')
#    dataset = dataset.register(workspace=ws,
#                                name=key,
#                                description=description_text)


#df = dataset.to_pandas_dataframe()
#df.describe()

found dataset


In [40]:
# dataset = ws.datasets[key]

In [8]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "iterations" : 2,
    "primary_metric" : 'r2_score'
}
project_folder = "./housing"
# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "regression",
                             training_data=dataset,
                             label_column_name="median_house_value",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [12]:
exp_automl = Experiment(ws, 'AutoML-housing')
automl_run = exp_automl.submit(automl_config, show_output = False)
RunDetails(automl_run).show()
automl_run.wait_for_completion()

Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_5fc93fb7-b188-4eff-be68-9a94ab6b8740',
 'target': 'compute-housing',
 'status': 'Completed',
 'startTimeUtc': '2021-02-06T07:21:49.899542Z',
 'endTimeUtc': '2021-02-06T07:32:12.274552Z',
 'properties': {'num_iterations': '2',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'r2_score',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'compute-housing',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoML-housing","subscription_id":"48a74bb7-9950-4cc1-9caa-5d50f995cc55","resource_group":"aml-quickstarts-137405","workspace_name":"quick-starts-ws-137405","region":"southcentralus","compute_target":"compute-housing","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":2,"primary_metric":"r2_score","task_type":"regression","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":

In [16]:
best_automl_run, best_automl_model = automl_run.get_output()
os.makedirs('./outputs_automl', exist_ok = True)
ml_path = r'./outputs_automl/model_best.joblib'
joblib.dump(value = best_automl_model, filename = ml_path)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


['./outputs_automl/model_best.joblib']

In [17]:
model_name = best_automl_run.properties['model_name']
automl_model = automl_run.register_model(model_name = model_name)

In [19]:
best_automl_run.get_file_names()

['automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_d49a3bb87614d8f552d254d5e25fd314670bbfbe738545a3037c25ddca21c78d_p.txt',
 'azureml-logs/65_job_prep-tvmps_d49a3bb87614d8f552d254d5e25fd314670bbfbe738545a3037c25ddca21c78d_p.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_d49a3bb87614d8f552d254d5e25fd314670bbfbe738545a3037c25ddca21c78d_p.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/104_azureml.log',
 'logs/azureml/azureml_automl.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py',
 'predicted_true',
 'residuals']

In [20]:
best_automl_run.download_file('outputs/conda_env_v_1_0_0.yml')
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py')

In [22]:
#create an inference configuration
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.from_conda_specification("myenv", 'conda_env_v_1_0_0.yml')
#env = Environment.get(workspace = ws, name = "AzureML-AutoML")

for pip_package in ["pandas","scikit-learn","joblib"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='scoring_file_v_1_0_0.py',
                                    environment=env)

In [23]:
from azureml.core.webservice import AciWebservice, AksWebservice, LocalWebservice
#deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

In [27]:
from azureml.core.webservice import LocalWebservice, Webservice
from azureml.core.model import InferenceConfig, Model

#deployment_config = LocalWebservice.deploy_configuration(port = 9000)
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, enable_app_insights = True)
first_model = Model(ws, name = model_name)
service = Model.deploy(ws, "automl-service", [first_model], inference_config, deployment_config, overwrite = True)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...............................

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#take a look at all the output metrics
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df_metrics = pd.DataFrame(deserialized_metrics_output)
df_metrics

In [ ]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

In [ ]:
print(best_model_output)

In [ ]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

In [ ]:
best_model.steps

In [ ]:
best_model

In [ ]:
type(best_model)

In [ ]:
#TODO: Save the best model
import joblib
os.makedirs('./outputs_automl', exist_ok = True)
ml_path = r'./outputs_automl/modelbest.joblib'
joblib.dump(value = best_model, filename=ml_path)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)